<a href="https://colab.research.google.com/github/valkova-k/cactus-repo/blob/br01/FashionMNIST_SimpleCNN_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
import random
import numpy as np

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**2. Data Preparation**

In [ ]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)    # smaller batch size

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 172kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.25MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.4MB/s]


**3. Neural Network Model**

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)    # added 3rd convolution layer
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=22*22*128, out_features=256)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=256, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
model

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=61952, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)    # added label smoothing for better regularization
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4, nesterov=True)    # added momentum (to speed up convergence), weight decay (penalizing large weights to reduce overfitting), nesterov momentum
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30], gamma=0.5)     # reduces lr during training, lr is multiplied by 0.5 after 10 epochs

**5. Training loop**

In [ ]:
for epoch in range(35):  # Train for 35 epochs
    running_loss = 0.0
    model.train()
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)    # prevent exploding gradients by limiting their norm to 1.0
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    print(f'Epoch [{epoch+1}/35], Loss: {running_loss / len(train_loader):.4f}, LR: {current_lr:.5f}')

Epoch [1/35], Loss: 0.9720, LR: 0.05000
Epoch [2/35], Loss: 0.7761, LR: 0.05000
Epoch [3/35], Loss: 0.7237, LR: 0.05000
Epoch [4/35], Loss: 0.6897, LR: 0.05000
Epoch [5/35], Loss: 0.6610, LR: 0.05000
Epoch [6/35], Loss: 0.6370, LR: 0.05000
Epoch [7/35], Loss: 0.6155, LR: 0.05000
Epoch [8/35], Loss: 0.5954, LR: 0.05000
Epoch [9/35], Loss: 0.5795, LR: 0.05000
Epoch [10/35], Loss: 0.5663, LR: 0.02500
Epoch [11/35], Loss: 0.5501, LR: 0.02500
Epoch [12/35], Loss: 0.5436, LR: 0.02500
Epoch [13/35], Loss: 0.5401, LR: 0.02500
Epoch [14/35], Loss: 0.5374, LR: 0.02500
Epoch [15/35], Loss: 0.5353, LR: 0.02500
Epoch [16/35], Loss: 0.5336, LR: 0.02500
Epoch [17/35], Loss: 0.5319, LR: 0.02500
Epoch [18/35], Loss: 0.5305, LR: 0.02500
Epoch [19/35], Loss: 0.5294, LR: 0.02500
Epoch [20/35], Loss: 0.5283, LR: 0.01250
Epoch [21/35], Loss: 0.5263, LR: 0.01250
Epoch [22/35], Loss: 0.5253, LR: 0.01250
Epoch [23/35], Loss: 0.5247, LR: 0.01250
Epoch [24/35], Loss: 0.5244, LR: 0.01250
Epoch [25/35], Loss: 0.52

**6. Evaluation on the test set**

In [ ]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 92.74%
